In [1]:
# --- Groq key -------------------------------------------------
API_KEY = "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"        #  <-- put real key
# -----------------------------------------------------------------------------


!pip -q install groq tqdm nest_asyncio

import os, re, json, random, asyncio, time, nest_asyncio
from tqdm import tqdm
from groq import AsyncGroq
nest_asyncio.apply()

if not API_KEY or API_KEY == "paste_your_groq_api_key_here":
    raise ValueError("❌  Replace API_KEY with your actual Groq key.")
os.environ["GROQ_API_KEY"] = API_KEY

# ---------- parameters ----------
MOTIONS      = [
    "Governments should ban fossil-fuel cars by 2035.",
    "Universal basic income should replace all means-tested welfare.",
]
NUM_MATCHES  = 6          # debates per pairing × motion
SEARCH_K     = 3          # candidates per turn for MCTS
MODEL_NAME   = "gemma2-9b-it"
SLEEP_SEC    = 2          # pause between debates (≈ 15 req/min incl. judge+score)
random.seed(42)

client = AsyncGroq(api_key=API_KEY)

# ---------- async helper ----------
async def gchat(messages, temp=0.8, max_tok=256):
    rsp = await client.chat.completions.create(
        model       = MODEL_NAME,
        messages    = messages,
        temperature = temp,
        max_tokens  = max_tok,
    )
    return rsp.choices[0].message.content.strip()

run = lambda coro: asyncio.get_event_loop().run_until_complete(coro)

# ---------- prompt builders ----------
def debater_prompt(side, motion):
    stance = "FOR" if side=="pro" else "AGAINST"
    return [
        {"role":"system", "content":
         f"You are a persuasive debater arguing {stance}:\n\"{motion}\""},
        {"role":"user",   "content":
         "Write ONE compelling sentence (≤25 words)."}
    ]

def scorer_prompt(sentence, side, motion):
    stance = "FOR" if side=="pro" else "AGAINST"
    return [
        {"role":"system", "content":"Rate persuasiveness 0-10; answer with one integer."},
        {"role":"user", "content":
         f"Motion: {motion}\nSentence: {sentence}\nRate for {stance} side."}
    ]

def score_sentence(sent, side, motion):
    out = run(gchat(scorer_prompt(sent, side, motion), temp=0, max_tok=8))
    m = re.search(r"\d+", out)
    return int(m.group()) if m else 5

JUDGE_SYS = (
 "You are the sole judge of a three-round debate.\n"
 'Return JSON: {"winner":"A|B|draw","score_A":0-10,"score_B":0-10,"reason":"<20 words>"}'
)
async def judge_async(text):
    msgs=[{"role":"system","content":JUDGE_SYS},
          {"role":"user","content":text}]
    for _ in range(2):
        raw = await gchat(msgs, temp=0, max_tok=96)
        m = re.search(r"\{.*\}", raw, re.S)
        if m:
            try: return json.loads(m.group())
            except: pass
        msgs += [{"role":"assistant","content":raw},
                 {"role":"user","content":"Please correct to valid JSON only."}]
    return {"winner":"draw","score_A":5,"score_B":5,"reason":"parse_fail"}
judge = lambda t: run(judge_async(t))

# ---------- agents ----------
def baseline(side, motion):
    return lambda *_: run(gchat(debater_prompt(side, motion), temp=0.9, max_tok=40))

def mcts(side, motion, k=SEARCH_K):
    def fn(_hist,_turn):
        best, best_s = "", -1
        for _ in range(k):
            cand = run(gchat(debater_prompt(side, motion), temp=1.2, max_tok=40))
            sc   = score_sentence(cand, side, motion)
            if sc > best_s: best_s, best = sc, cand
        return best
    return fn

# ---------- debate loop ----------
def play(motion, pro_bot, con_bot):
    log=[]
    for turn in range(3):
        log.append(f"A: {pro_bot(log,turn)}")
        log.append(f"B: {con_bot(log,turn)}")
    verdict = judge("\n".join(log))
    return verdict, log

# ---------- tournament ----------
rows=[]
for motion in MOTIONS:
    base_pro, base_con = baseline("pro",motion), baseline("con",motion)
    mcts_pro, mcts_con = mcts("pro",motion), mcts("con",motion)

    PAIRS = [("BASELINE vs BASELINE", base_pro, base_con),
             ("MCTS vs BASELINE",     mcts_pro, base_con),
             ("BASELINE vs MCTS",     base_pro, mcts_con)]

    for label,PRO,CON in PAIRS:
        wins=0
        print(f"{motion[:38]}… | {label} | {NUM_MATCHES} debates")
        for _ in tqdm(range(NUM_MATCHES), leave=False):
            ver,_ = play(motion, PRO, CON)
            wins += (ver["winner"]=="A")
            time.sleep(SLEEP_SEC)                   # stay within free rate limit
        rows.append((motion[:38]+"…"*(len(motion)>38), label, wins/NUM_MATCHES))

# ---------- results ----------
print("\n===== Win-rate summary =====")
for m,l,w in rows:
    print(f"{m:<40s} {l:<20s} {w:5.1%}")

print("\nSample debate – first motion (MCTS vs BASELINE)\n"+"-"*60)
v, smp = play(MOTIONS[0], mcts("pro", MOTIONS[0]), baseline("con", MOTIONS[0]))
print("\n".join(smp)); print("\nJudge:", v)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.2 MB/s eta 0:00:00
Governments should ban fossil-fuel car… | BASELINE vs BASELINE | 6 debates


Governments should ban fossil-fuel car… | MCTS vs BASELINE | 6 debates


Governments should ban fossil-fuel car… | BASELINE vs MCTS | 6 debates


Universal basic income should replace … | BASELINE vs BASELINE | 6 debates


Universal basic income should replace … | MCTS vs BASELINE | 6 debates


Universal basic income should replace … | BASELINE vs MCTS | 6 debates



===== Win-rate summary =====
Governments should ban fossil-fuel car…  BASELINE vs BASELINE 50.0%
Governments should ban fossil-fuel car…  MCTS vs BASELINE     100.0%
Governments should ban fossil-fuel car…  BASELINE vs MCTS     100.0%
Universal basic income should replace …  BASELINE vs BASELINE 33.3%
Universal basic income should replace …  MCTS vs BASELINE     50.0%
Universal basic income should replace …  BASELINE vs MCTS     33.3%

Sample debate – first motion (MCTS vs BASELINE)
------------------------------------------------------------
A: To save our planet and future generations, we must decisively transition to sustainable transportation by banning fossil-fuel cars now.
B: While ambitious, a 2035 ban ignores the technological advancements and infrastructure changes needed for a smooth transition to sustainable mobility.
A: Prioritizing a sustainable future by banning fossil-fuel cars ensures cleaner air, combats climate change, and secure a healthier planet for generations to